In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from os import listdir
from tensorflow.keras.preprocessing.image import load_img

In [2]:

plus_dir = 'C:\Dataset\+'
minus_dir = 'C:\Dataset\-'
times_dir = 'C:/Dataset/times'
plus_names = listdir(plus_dir)
minus_names = listdir(minus_dir)
times_names = listdir(times_dir)

In [3]:
plus_len = len([name for name in listdir(plus_dir)])
minus_len = len([name for name in listdir(minus_dir)])
times_len = len([name for name in listdir(times_dir)])

In [4]:
train_names = plus_names + minus_names + times_names
train_labels = []
for i in range(0, plus_len):
    train_labels.append('plus')
for i in range(plus_len, plus_len + minus_len):
    train_labels.append('minus')
for i in range(plus_len + minus_len, plus_len + minus_len + times_len):
    train_labels.append('times')
print(len(train_labels))

11434


In [5]:
for i in range(0, plus_len):
    train_names[i] = plus_dir + '/' + train_names[i]
for i in range(plus_len, plus_len + minus_len):
    train_names[i] = minus_dir + '/' + train_names[i]
for i in range(plus_len + minus_len, plus_len + minus_len + times_len):
    train_names[i] = times_dir + '/' + train_names[i]

In [6]:
train_df = pd.DataFrame({
    'id': train_names,
    'label': train_labels
})
train_df.head()

,id,label
0,C:\Dataset\+/exp12345.jpg,plus
1,C:\Dataset\+/exp12347.jpg,plus
2,C:\Dataset\+/exp12348.jpg,plus
3,C:\Dataset\+/exp12350.jpg,plus
4,C:\Dataset\+/exp12352.jpg,plus


In [29]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size = (3, 3),
                activation = 'relu',
                input_shape = (224, 224, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation = 'relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation = 'relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 109, 109, 32)      128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 109, 109, 32)     

In [19]:
data_gen_with_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

train_generator = data_gen_with_aug.flow_from_dataframe(
    train_df,
    x_col='id',
    y_col='label',
    class_mode = 'categorical',
    target_size = (224, 224),
    batch_size = 16
)

Found 11434 validated image filenames belonging to 3 classes.


In [30]:
import sys
from PIL import Image
sys.modules['Image'] = Image 
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit_generator(
    train_generator,
    epochs = 10,
    shuffle = True,
    steps_per_epoch = 3)


  ...
    to  
  ['...']
Train for 3 steps
Epoch 1/10
3/3 [==============================] - 7s 2s/step - loss: 1.1165 - accuracy: 0.6042
Epoch 2/10
3/3 [==============================] - 5s 2s/step - loss: 0.8440 - accuracy: 0.7708
Epoch 3/10
3/3 [==============================] - 6s 2s/step - loss: 0.7272 - accuracy: 0.7500
Epoch 4/10
3/3 [==============================] - 6s 2s/step - loss: 0.4865 - accuracy: 0.8333
Epoch 5/10
3/3 [==============================] - 6s 2s/step - loss: 0.4314 - accuracy: 0.8750
Epoch 6/10
3/3 [==============================] - 6s 2s/step - loss: 0.5697 - accuracy: 0.8333
Epoch 7/10
3/3 [==============================] - 6s 2s/step - loss: 0.5201 - accuracy: 0.8333
Epoch 8/10
3/3 [==============================] - 6s 2s/step - loss: 0.1255 - accuracy: 0.9375
Epoch 9/10
3/3 [==============================] - 5s 2s/step - loss: 0.2050 - accuracy: 0.9583
Epoch 10/10
3/3 [==============================] - 5s 2s/step - loss: 0.1976 - accuracy: 0.9167
